## Попробуем поучиться на разных наборах фич

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from notebooks.helpers import explore_data_modern
from catboost import Pool,CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

Настройки отображения

In [2]:
pd.set_option('display.max_columns', 50)
plt.style.use('ggplot') 

Загрузка данных

In [3]:
try:
    X_TRAIN = pd.read_parquet('../data/datasets/train.parquet')
    X_VAL = pd.read_parquet('../data/datasets/val.parquet')
        
    print("✅ Данные загружены!")
    print(f"Train: {X_TRAIN.shape[0]} строк, Validation: {X_VAL.shape[0]} строк")
except Exception as e:
    print(f"❌ Ошибка: {e}")

✅ Данные загружены!
Сессии: 1860042 строк, 18 колонок
События: 15726470 строк, 11 колонок


In [55]:
display(explore_data_modern(X_TRAIN, 'Train'))


🔍 Анализ датафрейма: Train


In [63]:
def learn(candidate):
    for feature_set in candidate:
        print(f"Feature set {feature_set}")
    best_auc = 0
    best_features = []
    for feature_set in candidate:
        X_train = X_TRAIN[feature_set]
        y_train = X_TRAIN['target']
        y_val = X_VAL['target']
        X_val = X_VAL[feature_set]
        model = CatBoostClassifier(
            cat_features=[f for f in feature_set if X_train[f].dtype == 'object'],
            auto_class_weights='Balanced',
            verbose=0
        )
        model.fit(X_train, y_train)
    
        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
    
        if auc > best_auc:
            best_auc = auc
            best_features = feature_set
        print(f"New best ROC-AUC: {auc:.3f} | Features: {feature_set}")

In [147]:
temp_features = [
    ['utm_keyword'],
    ['utm_adcontent'],
    ['utm_campaign'],
    ['frequent_visitor'],
    ['brand_tier'],
]

In [148]:
learn(temp_features)

Feature set ['utm_keyword']
Feature set ['utm_adcontent']
Feature set ['utm_campaign']
Feature set ['frequent_visitor']
Feature set ['brand_tier']
New best ROC-AUC: 0.561 | Features: ['utm_keyword']
New best ROC-AUC: 0.593 | Features: ['utm_adcontent']
New best ROC-AUC: 0.627 | Features: ['utm_campaign']
New best ROC-AUC: 0.530 | Features: ['frequent_visitor']
New best ROC-AUC: 0.523 | Features: ['brand_tier']


In [149]:
candidate_features = [
    ['utm_source', 'utm_medium', 'device_brand'],
    ['utm_source', 'utm_medium', 'brand_tier'],
    ['utm_source', 'utm_medium', 'visit_number'],
    ['utm_source', 'utm_medium', 'frequent_visitor'],
]

In [150]:
learn(candidate_features)

Feature set ['utm_source', 'utm_medium', 'device_brand']
Feature set ['utm_source', 'utm_medium', 'brand_tier']
Feature set ['utm_source', 'utm_medium', 'visit_number']
Feature set ['utm_source', 'utm_medium', 'frequent_visitor']
New best ROC-AUC: 0.674 | Features: ['utm_source', 'utm_medium', 'device_brand']
New best ROC-AUC: 0.666 | Features: ['utm_source', 'utm_medium', 'brand_tier']
New best ROC-AUC: 0.667 | Features: ['utm_source', 'utm_medium', 'visit_number']
New best ROC-AUC: 0.664 | Features: ['utm_source', 'utm_medium', 'frequent_visitor']


In [151]:
candidate_features = [
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number'],
    ['utm_source', 'utm_medium', 'brand_tier', 'visit_number']
]

In [152]:
learn(candidate_features)

Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number']
Feature set ['utm_source', 'utm_medium', 'brand_tier', 'visit_number']
New best ROC-AUC: 0.682 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number']
New best ROC-AUC: 0.675 | Features: ['utm_source', 'utm_medium', 'brand_tier', 'visit_number']


In [153]:
candidate_features = [
    ['utm_source', 'utm_campaign' ],
    ['utm_source', 'utm_medium', 'utm_campaign'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign'],
    ['utm_source', 'utm_medium', 'brand_tier', 'visit_number', 'utm_campaign']
]

In [154]:
learn(candidate_features)

Feature set ['utm_source', 'utm_campaign']
Feature set ['utm_source', 'utm_medium', 'utm_campaign']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign']
Feature set ['utm_source', 'utm_medium', 'brand_tier', 'visit_number', 'utm_campaign']
New best ROC-AUC: 0.668 | Features: ['utm_source', 'utm_campaign']
New best ROC-AUC: 0.672 | Features: ['utm_source', 'utm_medium', 'utm_campaign']
New best ROC-AUC: 0.694 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign']
New best ROC-AUC: 0.687 | Features: ['utm_source', 'utm_medium', 'brand_tier', 'visit_number', 'utm_campaign']


In [155]:
candidate_features = [
    ['utm_source', 'utm_adcontent'],
    ['utm_source', 'utm_medium', 'utm_adcontent'],
    ['utm_source', 'utm_medium', 'device_brand', 'utm_adcontent'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_adcontent'],
]

In [156]:
learn(candidate_features)

Feature set ['utm_source', 'utm_adcontent']
Feature set ['utm_source', 'utm_medium', 'utm_adcontent']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'utm_adcontent']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_adcontent']
New best ROC-AUC: 0.652 | Features: ['utm_source', 'utm_adcontent']
New best ROC-AUC: 0.663 | Features: ['utm_source', 'utm_medium', 'utm_adcontent']
New best ROC-AUC: 0.679 | Features: ['utm_source', 'utm_medium', 'device_brand', 'utm_adcontent']
New best ROC-AUC: 0.687 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_adcontent']


In [159]:
candidate_features = [
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_adcontent']
]

In [160]:
learn(candidate_features)

Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_adcontent']
New best ROC-AUC: 0.694 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_adcontent']


In [161]:
candidate_features = [
    ['utm_source', 'utm_keyword'],
    ['utm_source', 'utm_medium', 'utm_keyword'],
    ['utm_source', 'utm_medium', 'device_brand', 'utm_keyword'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_keyword'],
]

In [162]:
learn(candidate_features)

Feature set ['utm_source', 'utm_keyword']
Feature set ['utm_source', 'utm_medium', 'utm_keyword']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'utm_keyword']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_keyword']
New best ROC-AUC: 0.651 | Features: ['utm_source', 'utm_keyword']
New best ROC-AUC: 0.660 | Features: ['utm_source', 'utm_medium', 'utm_keyword']
New best ROC-AUC: 0.675 | Features: ['utm_source', 'utm_medium', 'device_brand', 'utm_keyword']
New best ROC-AUC: 0.685 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_keyword']


In [169]:
candidate_features = [
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword', 'device_category']
]

In [170]:
learn(candidate_features)

Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword', 'device_category']
New best ROC-AUC: 0.699 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword', 'device_category']


In [58]:
candidate_features = [
    ['utm_source'],
    ['utm_source', 'utm_medium'],
    ['utm_source', 'utm_medium', 'device_brand'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword'],
]

In [59]:
learn(candidate_features)

Feature set ['utm_source']
Feature set ['utm_source', 'utm_medium']
Feature set ['utm_source', 'utm_medium', 'device_brand']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword']
New best ROC-AUC: 0.638 | Features: ['utm_source']
New best ROC-AUC: 0.654 | Features: ['utm_source', 'utm_medium']
New best ROC-AUC: 0.673 | Features: ['utm_source', 'utm_medium', 'device_brand']
New best ROC-AUC: 0.683 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number']
New best ROC-AUC: 0.693 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign']
New best ROC-AUC: 0.700 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword']


In [33]:
candidate_features = [
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword_notset_fill'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword_campaign_fill'],
    ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'has_utm_keyword'],
]

In [34]:
learn(candidate_features)

Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword_notset_fill']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword_campaign_fill']
Feature set ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'has_utm_keyword']
New best ROC-AUC: 0.701 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword']
New best ROC-AUC: 0.701 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword_notset_fill']
New best ROC-AUC: 0.699 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword_campaign_fill']
New best ROC-AUC: 0.700 | Features: ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'has_utm_keyword']


In [64]:
candidate_features = [
    ['utm_source']
]

In [65]:
learn(candidate_features)

Feature set ['utm_source']
New best ROC-AUC: 0.638 | Features: ['utm_source']


In [ ]:
del X_VAL, X_TRAIN